__Demultiplexing the two sequencing libraries using barcode splitter and then trimmatic(?). Following Michelle's demultiplexing steps on https://github.com/pinskylab/genomics/blob/df5c254a0e097da18f49532f748a5a2ae5e1299f/Protocols/01_hiseq_workflow.md as a rough guide, and barcode_splitter.py documentation: https://pypi.org/project/barcode-splitter/__

__Start with sequencing library 1__

In [1]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing


__Each capture (1-16) was a PCR pool, first separate out by PCR pool. First, in SP1__

In [2]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1


__Find which files the barcodes are in, use ATCACG as the example barcode__

In [6]:
!zcat GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz | grep -c "^ATCACG"

4


In [8]:
!zcat GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | grep -c "^ATCACG"

22910439


__Look at the head of the compressed file to see where the barcode position is relative to index__

In [12]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | grep "^ATCACG" | head

ATCACGCACATGAGGACATCAACAGGAACTGACTAGTAAGTCACCTCAGGTGCTGAGGATT
ATCACGTCGCCTGCATAATTAGCATTTTATTGTCAAGTGTGCGAGCTAAGTCAGTTAATGA
ATCACGAAAACGATTTCAAACTTGGAATAAATGGCTCAGGACTTGTCTAAAATTAATCAAA
ATCACGACCTGTTTGTCTCTGAGAGATAGAGACACCACCTGGATTGCAAGGTTGTCCTGCA
ATCACGAACACTACTTGATGTAATTTATTAATTTGCAAAATTTTCAGCAGTTACAAAAAAA
ATCACGTCCGTTCGCTAATACCTGGAGGCATGCAGAAGAATAATTTACACTAGTGAGATGA
ATCACGAGTTACATTTTAGGTCTCTTTTTTTCCTTTTTTCAGTGATGAGAAAGAAAACAAA
ATCACGTAGGAGTCACCTTGTGCAGCGATACACCTCTTCCAGCACTGTTATCAAGCTTGTA
ATCACGTCTTCAGTCAAAAAAAGAAAAGTAACAAGTGGAAAGAGCAAATGTGAACATCTTT
ATCACGAGACTTCCAATTCAAACGCTCCCACAGTAGCTCTGCTAATGACCCAGAGGGCTGT
grep: write error

gzip: stdout: Broken pipe


In [5]:
!gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_4_passed_filter.fastq.gz | grep "^CATCGA" | head #reverse compliment of the read one barcode

CATCGAGTCGACTTGCGTAAAGAATACCTTCTAAGAATAGTAGTCAGAATCCTCCACAAAT
CATCGACTCGACTTCAGCAGTTCAGACAGCACTGGCTCCACTTCCCCTGCCACCACACCAA
CATCGAGTCGACTTCATCTCTGGATGGCATGGAGACAACATGCTGGAGACCAGTGAAAAGA
CATCGAGTCGACTTCAGGACTTCACCCACCTTCCGTCGCCCTAAAACCCTCCGTCTCCGCA
CATCGAGTCGACTTCAACTTCTACAAGAACTGGCTGGGGCTCGGCCTCTACGAGCGCGACC
CATCGAGTCGACTTGTGCAGATCACTGACCCAGACGGGAAGCCGAAGAAGGCCAACATCCG
CATCGAGTCGACTTGAAACATCCACAGTACACTCTGGTGGTACAAGCAGCAGACATGGTAG
CATCGAGTCGACTTCGGACGGAGTCGCCGTGCTCAAGGTGGGAGGAACCAGTGATGTGGAG
CATCGAGTCGACTTATCCTCTCCTGCCTCGCCTTCGCCGGCGCCGCCTACGGTTGCGGTAA
CATCGACTCGACTTTTTTTCTATGATGATTTGTACGTTGCTTGTGTGGAAAAATTTAGGGT
grep: write error

gzip: stdout: Broken pipe


__Okay it looks like the barcode is to the right of the index, left of the read as expected for the forward and reverse reads, and in reverse read the sequence is reversed__

__Where is the PCR index? Should be beginning of index read, use ATTACTCG as example__

In [ ]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz | grep "^ATTACTCG" | head

In [9]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | head

@A00741:178:H55NJDRXY:2:2101:2211:1000 1:N:0:0
GATCAGAAACCACCTTCTGATACCTGAAATCTCTGAGTTTTCACACAAAGAACACATGAAC
+
FFFFF:FFFFF:FFFF:,:FFF,FFFFFFFF:FFFFFFFF:F:,F,FFF,FFFF,,F,:FF
@A00741:178:H55NJDRXY:2:2101:2230:1000 1:N:0:0
CGATGTAGAGATTGATAATGTTTGATATTTTCACCACAGTAAAAGAAAGACAGCAACACCC
+
FFFFFFFFFFF:::,:F,:FFF:FFFFF,,,FFFF,FFFFFFFFF::FFFFF:FF:,F:FF
@A00741:178:H55NJDRXY:2:2101:2266:1000 1:N:0:0
CAGATCATGTTAGCTTAGCTTAGCCTAGCTTAGCATAAACAAAGAAGGTCACAACTGTTAG

gzip: stdout: Broken pipe


In [6]:
 !gzip -cd GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_4_passed_filter.fastq.gz | head

@A00741:178:H55NJDRXY:2:2101:2211:1000 4:N:0:0
GNTCAGATATAAAATGTGAACTACTTGTTGGGTTCTCTCTGTTCTCATGTATCTTACATGT
+
F#FF,:,,FFFF,FFF:::,,:,F:,FF:,,F,,FFF:FFFF,FFFF,F,,F,FFF,FF,:
@A00741:178:H55NJDRXY:2:2101:2230:1000 4:N:0:0
CGATGTTAATGGCATTTTTATAGCAAGTATTTAATTGAGTTTCAATAAGAAAAGGAAATAT
+
FFFFFF,:FFF,FFFF:F,FFFFFF,FFFFFFFFFFFFFFF::FF,FF:F:FF,FFF::FF
@A00741:178:H55NJDRXY:2:2101:2266:1000 4:N:0:0
CAGATCAAGCTAGGCTAAGCTAAGCTAACAGTTCTGACCTTTTCTCCGTCTTTATGCTAAG

gzip: stdout: Broken pipe


__Count the number of reads__

In [2]:
!zcat GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz | wc -l | awk '{print$1/4}'
!zcat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_1_passed_filter.fastq.gz  | wc -l | awk '{print$1/4}'

gzip: GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz: No such file or directory
0
536138868


__Run barcode splitter in a screen on annotate__

In [8]:
#run within screen annotate
!/local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/paired_sequence_utils/barcode_splitter.py --bcfile /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/PCRCapturePoolsSP1.tsv /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_1_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_2_Index_Read_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/GCF-KC-4172-SP1-for-138-cycles-H55NJDRXY_2_Read_4_passed_filter.fastq.gz --idxread 2  

__Now for SP2__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/bcsplit


In [ ]:
#run within screen annotate or make sure this notebook is in a screen
!/local/home/katrinac/ClownfishGWAS/bioinformatics_scripts/paired_sequence_utils/barcode_splitter.py --bcfile /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/PCRCapturePoolsSP2.tsv /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_1_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_2_Index_Read_passed_filter.fastq.gz /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/GCF-KC-4172-SP2-for-138-cycles-H5MVVDRXY_1_Read_4_passed_filter.fastq.gz --idxread 2  

__Now split out by barcoded adapter sequence, remove adapters, and trime low quality reads with ultraplex https://github.com/ulelab/ultraplex__

In [1]:
#move to subdirectory
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/


/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in


In [ ]:
!#January 25, there's still barcodes in the beginning of the read 2 files after fastp adapter trimming. I'm going to try ultraplex instead.
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_03_26_SEQ1/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/
cp ~/ClownfishGWAS/data/APPC_sequencing/novaseq_2021_04_16_SEQ2/bcsplit/*.gz ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/


In [ ]:
#not sure why doing this in a bash for loop fails, but it's not worth figuring it out just run each capture pool separately
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP1-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP1_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP1 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP2-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP2-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP2_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP2 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP3-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP3-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP3_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP3 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP4-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP4-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP4_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP4 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP5-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP5-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP5_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP5 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP6-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP6-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP6_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP6 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP7-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP7-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP7_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP7 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP8-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP8-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP8_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP8 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP9-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP9-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP9_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP9 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP10-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP10-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP10_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP10 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP11-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP11-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP11_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP11 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP12-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP12-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP12_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP12 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP13-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP13-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP13_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP13 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP14-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP14-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP14_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP14 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP15-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP15-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP15_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP15 
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP16-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP16-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP16_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP16 
#failed ultraplex bash loop
#for i in $(cat /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/cap_filenames);      do ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/$i-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/$i_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/ultraplex_out -o $i;      don


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@   @@@@   .@@   @@@@@@@          @@         @@@@@@    (@@@@@        @@@    @@@@@@@         @@    @@@    @
@@@@   @@@@    @@   ,@@@@@@@@@    @@@@@    @@@   @@@@   (   @@@@   @@@   @@    @@@@@@@   @@@@@@@@@@   #   @@@@
@@@   &@@@    @@    @@@@@@@@@%   @@@@@         @@@@(   @    @@@         @@    @@@@@@@         @@@@@     @@@@@@
@@    @@@    @@    @@@@@@@@@@   @@@@@    @@    @@@          @@   .@@@@@@@*   @@@@@@@    @@@@@@@@.   @    @@@@@
@@@       @@@@.        @@@@@   @@@@@&   @@@.   &   &@@@@    @    @@@@@@@@         @          @    @@@@    @@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

Namespace(adapter='AGATCGGAAGAGCGGTTCAG', adapter2='AGATCGGAAGAGCGTCGTG', barcodes='/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP1_barcodes.csv', directory='/local/home/katrinac/ClownfishGWAS/da

Demultiplexing...
1 million reads processed
2 million reads processed
3 million reads processed
4 million reads processed
5 million reads processed
6 million reads processed
7 million reads processed
8 million reads processed
9 million reads processed
10 million reads processed
11 million reads processed
12 million reads processed
13 million reads processed
14 million reads processed
15 million reads processed
16 million reads processed
17 million reads processed
18 million reads processed
19 million reads processed
20 million reads processed
21 million reads processed
22 million reads processed
23 million reads processed
24 million reads processed
25 million reads processed
26 million reads processed
27 million reads processed
28 million reads processed
29 million reads processed
30 million reads processed
31 million reads processed
32 million reads processed
33 million reads processed
34 million reads processed
35 million reads processed
36 million reads processed
37 million reads pr

11 million reads processed
12 million reads processed
13 million reads processed
14 million reads processed
15 million reads processed
16 million reads processed
17 million reads processed
18 million reads processed
19 million reads processed
20 million reads processed
21 million reads processed
22 million reads processed
23 million reads processed
24 million reads processed
25 million reads processed
26 million reads processed
27 million reads processed
28 million reads processed
29 million reads processed
30 million reads processed
31 million reads processed
32 million reads processed
33 million reads processed
34 million reads processed
35 million reads processed
36 million reads processed
37 million reads processed
38 million reads processed
39 million reads processed
40 million reads processed
41 million reads processed
42 million reads processed
43 million reads processed
44 million reads processed
45 million reads processed
46 million reads processed
47 million reads processed
4

70 million reads processed
71 million reads processed
72 million reads processed
73 million reads processed
74 million reads processed
75 million reads processed
76 million reads processed
77 million reads processed
78 million reads processed
79 million reads processed
80 million reads processed
81 million reads processed
82 million reads processed
83 million reads processed
84 million reads processed
Demultiplexing complete! 85811687 reads processed in 1099.0 seconds

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@   @@@@   .@@   @@@@@@@          @@         @@@@@@    (@@@@@        @@@    @@@@@@@         @@    @@@    @
@@@@   @@@@    @@   ,@@@@@@@@@    @@@@@    @@@   @@@@   (   @@@@   @@@   @@    @@@@@@@   @@@@@@@@@@   #   @@@@
@@@   &@@@    @@    @@@@@@@@@%   @@@@@         @@@@(   @    @@@         @@    @@@@@@@         @@@@@     @@@@@@
@@    @@@    @@    @@@@@@@@@@   @@@@@    @@    @@@          @@   .@@@@@@@*   @@@@@

Demultiplexing...
1 million reads processed
2 million reads processed
3 million reads processed
4 million reads processed
5 million reads processed
6 million reads processed
7 million reads processed
8 million reads processed
9 million reads processed
10 million reads processed
11 million reads processed
12 million reads processed
13 million reads processed
14 million reads processed
15 million reads processed
16 million reads processed
17 million reads processed
18 million reads processed
19 million reads processed
20 million reads processed
21 million reads processed
22 million reads processed
23 million reads processed
24 million reads processed
25 million reads processed
26 million reads processed
27 million reads processed
28 million reads processed
29 million reads processed
30 million reads processed
31 million reads processed
32 million reads processed
33 million reads processed
34 million reads processed
35 million reads processed
36 million reads processed
37 million reads pr

In [8]:
!ultraplex -i /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP16-read-1.fastq.gz -i2 /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_in/CAP16-read-3.fastq.gz -b /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP16_barcodes.csv -t 8 -d /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out -o CAP16 



@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@   @@@@   .@@   @@@@@@@          @@         @@@@@@    (@@@@@        @@@    @@@@@@@         @@    @@@    @
@@@@   @@@@    @@   ,@@@@@@@@@    @@@@@    @@@   @@@@   (   @@@@   @@@   @@    @@@@@@@   @@@@@@@@@@   #   @@@@
@@@   &@@@    @@    @@@@@@@@@%   @@@@@         @@@@(   @    @@@         @@    @@@@@@@         @@@@@     @@@@@@
@@    @@@    @@    @@@@@@@@@@   @@@@@    @@    @@@          @@   .@@@@@@@*   @@@@@@@    @@@@@@@@.   @    @@@@@
@@@       @@@@.        @@@@@   @@@@@&   @@@.   &   &@@@@    @    @@@@@@@@         @          @    @@@@    @@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

Namespace(adapter='AGATCGGAAGAGCGGTTCAG', adapter2='AGATCGGAAGAGCGTCGTG', barcodes='/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/multiplexing/CAP16_barcodes.csv', directory='/local/home/katrinac/ClownfishGWAS/d

__rename ultraplex output files__

In [9]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out
!rename 'ultraplex_' '' *.gz #delete the ultraplex prefix
!rename 'Fwd' 'R1' *.gz #rename to read 1
!rename 'Rev' 'R2' *.gz #rename to read 2

/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out


__copy the de-multiplexed data to a fresh directory for fastq screen__

In [11]:
#!mkdir ~/ClownfishGWAS/data/APPC_sequencing/fastq_screen
!cp ~/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/$(ls *.gz -I '*no_match*') /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/ultraplex_out/fastq_screen #only move the matched files with the .gz extension to leave behind the logs

__move to qc_seq.ipynb__